In [1]:
######################################### Get file & load into json #########################################
import json
import pandas as pd
import requests
import json
from datetime import date
from datetime import datetime, timedelta
from pandas.io.json import json_normalize
from flatten_json import flatten

url_API = "https://api.battlepeer.com/organizations/api-key/organizations?apiKey=pF57WF2rmnDkoayOXdtLYTFYRM"
# url_api = "/articles?include=author&fields[articles]=title,body&fields[people]=name"
response_API = requests.get(url_API)
# print(response_API.status_code)

data = response_API.text
parse_json = json.loads(data)
# print("Survey: ", parse_json)

yFile = json.dumps(parse_json, indent=4)
print(yFile)

[
    {
        "id": 3,
        "name": "Great Lakes Dryhootch",
        "address": "1030 E Brady St, Milwaukee, WI 53202",
        "users": [
            {
                "id": 43,
                "initials": "Z.F.",
                "checkIn": [
                    {
                        "date": "2023-06-24T20:32:09.108Z",
                        "max_score": 15,
                        "score": 13,
                        "ratio": 0.87,
                        "ratioDisplayName": "good"
                    }
                ]
            },
            {
                "id": 45,
                "initials": "M.L.",
                "checkIn": [
                    {
                        "date": "2023-01-23T17:48:02.183Z",
                        "max_score": 15,
                        "score": 11,
                        "ratio": 0.73,
                        "ratioDisplayName": "normal"
                    },
                    {
                        "date": "2023-01-24T

In [2]:
##################################### Get json written into a excel file, add the name Survey #####################################

from datetime import date
import os

today_date = date.today()
print(today_date)
# mypath_short = "C:/Users/lucyle/Desktop/BattlePeer_json/" + str(today_date) + "/"
# mypath_short = "/Users/lucyle/Desktop/2022 PhD CompScience/iPeer Veterans w PTSD Research/BattlePeer json/"
temp_path = "/Users/lucyl/Desktop/2022_PhD_CompScience/BattlePeer_Research/BattlePeer_Data"
mypath_short = temp_path + "/" + str(today_date) + "/"

# mode = 0o666
os.mkdir(mypath_short)

my_string =  "iPeer_API_" + str(today_date) + ".json"
mypath = mypath_short + my_string
print(mypath)

# write the text data to json file
with open(mypath, 'w') as f:
    f.writelines("{ \"survey\": ") # name of json data
    f.writelines(yFile)
    f.writelines("}")    


2023-07-27
/Users/lucyl/Desktop/2022_PhD_CompScience/BattlePeer_Research/BattlePeer_Data/2023-07-27/iPeer_API_2023-07-27.json


In [3]:
##################################### explore json file and normalize it ##################################### 
# !pip3 install openpyxl

# load text data 
with open(mypath) as f:
    d = json.load(f)
    # print(d)


# normalize and flatten file/data frame
df = pd.json_normalize(d, record_path =['survey'])
print("size: ",df.shape)

df = df.explode('users')
df = pd.json_normalize(json.loads(df.to_json(orient="records")))
newpath = mypath_short + str(today_date) + "_df.xlsx"
df.to_excel(newpath)

# print(df)

size:  (11, 4)


In [4]:
######################################### explode users more with details #########################################

df = df.explode('users.checkIn')
df = pd.json_normalize(json.loads(df.to_json(orient="records")))
# df2 = df.drop('users', axis=1) #drop column users
newpath = mypath_short + str(today_date) + "_df_users.xlsx"
# df.to_excel(newpath)
print(df)


      id                        name                               address  \
0      3       Great Lakes Dryhootch  1030 E Brady St, Milwaukee, WI 53202   
1      3       Great Lakes Dryhootch  1030 E Brady St, Milwaukee, WI 53202   
2      3       Great Lakes Dryhootch  1030 E Brady St, Milwaukee, WI 53202   
3      3       Great Lakes Dryhootch  1030 E Brady St, Milwaukee, WI 53202   
4      3       Great Lakes Dryhootch  1030 E Brady St, Milwaukee, WI 53202   
...   ..                         ...                                   ...   
1024   6  Organization name for test         775 Westminster Avenue APT D5   
1025   6  Organization name for test         775 Westminster Avenue APT D5   
1026   8          First organization         775 Westminster Avenue APT D5   
1027  10           some_organization                     some_organization   
1028  11           Organization name                  Organization address   

      users.id users.initials users        users.checkIn.date  

In [5]:
############################# get the date extracted from users.checkIn.dates #############################

# get index of column with date
index1 = -1
for col in df.columns:
    if "checkIn.date" in col:
        # print(col)
        index1 = df.columns.get_loc(col)
print(index1)

# Check if successfully get the date column correctly
if index1 == -1:
    print("Failed to get index. Exit....!")
    sys.exit()

# Add the dates into a new column after extracting/splitting string 
dates_list = []
for date in df.iloc[:,index1]:
    mystr = str(date)
    if mystr == "":
        dates_list.append("nan")
    else:
        mystr = str(date).split("T")
        date = mystr[0]
        dates_list.append(date)
        
for i in range(0, len(dates_list)):
    if dates_list[i] == "None":
        dates_list[i] = "nan"

df["Short Date"] = dates_list

newpath = mypath_short + str(today_date) + "_df_dates.xlsx"
# df.to_excel(newpath)

print(df['Short Date'])

6
0       2023-06-24
1       2023-01-23
2       2023-01-24
3       2023-01-30
4       2023-01-31
           ...    
1024    2022-11-09
1025           nan
1026           nan
1027           nan
1028           nan
Name: Short Date, Length: 1029, dtype: object


In [6]:
############################# Get week numbers and add it into new column #############################
############################# Take only dates starting from 2023-1-1 #############################
from datetime import *

weeks_list = []

date_2023 = date(2023,1,1)

for date in df['Short Date']:
    if date != 'nan':
        date_object = datetime.strptime(date, '%Y-%m-%d').date()
        week_number = date_object.isocalendar()[1]

        if date_object < date_2023:
            # date_object = "nan"
            week_number = 0

        weeks_list.append(week_number)
    else:
        weeks_list.append("nan")
        
df["Week Number"] = weeks_list

newpath = mypath_short + str(today_date) + "_df_weeks.xlsx"

# df.to_excel(newpath)


In [7]:
############################# Drop rows with dates less than 2023-1-1 #############################

df2 = df[df["Week Number"] != 0]
newpath = mypath_short + str(today_date) + "_df_new.xlsx"
# df2.to_excel(newpath)


In [8]:
############################ Get individual dataset for each mentee & put in excel ###################################
import math
mentors_list = df2["id"].unique()
print("Mentors ID: ", mentors_list)

mentors_list = df2["name"].unique()
print("Mentor Organizations: ", mentors_list)

df3 = pd.DataFrame
df3 = df2[df2["id"] == 3]
 
mentees_list = df3["users.id"].unique()   

print(mentees_list.shape[0])

list_menteedf = []
# print(mypath_short) 
newfile = mypath_short + "Mentees/"
print(newfile)
os.mkdir(newfile)

for id in mentees_list:
    df_mentee = pd.DataFrame
    df_mentee = df3[df3["users.id"] == id]
    list_menteedf.append(df_mentee)
    s_name = newfile + str(id).replace(".0", "") + ".xlsx"
    print(s_name)
    df_mentee.to_excel(s_name) 



Mentors ID:  [ 3  2  5 14 12  7  6  8 10 11]
Mentor Organizations:  ['Great Lakes Dryhootch' 'Organization for tests'
 'Great Lakes Dryhootch Inc.' 'Bob Curry Test Organization' 'Test Duncie'
 'Test Organization III' 'Organization name for test' 'First organization'
 'some_organization' 'Organization name']
87
/Users/lucyl/Desktop/2022_PhD_CompScience/BattlePeer_Research/BattlePeer_Data/2023-07-27/Mentees/
/Users/lucyl/Desktop/2022_PhD_CompScience/BattlePeer_Research/BattlePeer_Data/2023-07-27/Mentees/43.xlsx
/Users/lucyl/Desktop/2022_PhD_CompScience/BattlePeer_Research/BattlePeer_Data/2023-07-27/Mentees/45.xlsx
/Users/lucyl/Desktop/2022_PhD_CompScience/BattlePeer_Research/BattlePeer_Data/2023-07-27/Mentees/46.xlsx
/Users/lucyl/Desktop/2022_PhD_CompScience/BattlePeer_Research/BattlePeer_Data/2023-07-27/Mentees/50.xlsx
/Users/lucyl/Desktop/2022_PhD_CompScience/BattlePeer_Research/BattlePeer_Data/2023-07-27/Mentees/52.xlsx
/Users/lucyl/Desktop/2022_PhD_CompScience/BattlePeer_Research/Bat

In [9]:
################################# Add all mentee files in dataframe #################################
import os
import openpyxl

print(mypath_short)
folder = mypath_short + "Mentees/"
print(folder)

dir_list = os.listdir(folder)
# print("Files and directories in '", folder, "' :")
# print(dir_list)

df_master = []
for file in dir_list:
    filename = folder + file 
    print(filename)
    df = pd.read_excel(filename)
    df_master.append(df)

print(df_master[1])

/Users/lucyl/Desktop/2022_PhD_CompScience/BattlePeer_Research/BattlePeer_Data/2023-07-27/
/Users/lucyl/Desktop/2022_PhD_CompScience/BattlePeer_Research/BattlePeer_Data/2023-07-27/Mentees/
/Users/lucyl/Desktop/2022_PhD_CompScience/BattlePeer_Research/BattlePeer_Data/2023-07-27/Mentees/58.xlsx
/Users/lucyl/Desktop/2022_PhD_CompScience/BattlePeer_Research/BattlePeer_Data/2023-07-27/Mentees/100.xlsx
/Users/lucyl/Desktop/2022_PhD_CompScience/BattlePeer_Research/BattlePeer_Data/2023-07-27/Mentees/116.xlsx
/Users/lucyl/Desktop/2022_PhD_CompScience/BattlePeer_Research/BattlePeer_Data/2023-07-27/Mentees/141.xlsx
/Users/lucyl/Desktop/2022_PhD_CompScience/BattlePeer_Research/BattlePeer_Data/2023-07-27/Mentees/62.xlsx
/Users/lucyl/Desktop/2022_PhD_CompScience/BattlePeer_Research/BattlePeer_Data/2023-07-27/Mentees/74.xlsx
/Users/lucyl/Desktop/2022_PhD_CompScience/BattlePeer_Research/BattlePeer_Data/2023-07-27/Mentees/54.xlsx
/Users/lucyl/Desktop/2022_PhD_CompScience/BattlePeer_Research/BattlePeer_D

In [10]:
# Build date object list for all dates until today
# current_date = datetime.today()
datelist_2023 = pd.date_range(start="2023-01-01",end=today_date).tolist()
print(datelist_2023)

[Timestamp('2023-01-01 00:00:00', freq='D'), Timestamp('2023-01-02 00:00:00', freq='D'), Timestamp('2023-01-03 00:00:00', freq='D'), Timestamp('2023-01-04 00:00:00', freq='D'), Timestamp('2023-01-05 00:00:00', freq='D'), Timestamp('2023-01-06 00:00:00', freq='D'), Timestamp('2023-01-07 00:00:00', freq='D'), Timestamp('2023-01-08 00:00:00', freq='D'), Timestamp('2023-01-09 00:00:00', freq='D'), Timestamp('2023-01-10 00:00:00', freq='D'), Timestamp('2023-01-11 00:00:00', freq='D'), Timestamp('2023-01-12 00:00:00', freq='D'), Timestamp('2023-01-13 00:00:00', freq='D'), Timestamp('2023-01-14 00:00:00', freq='D'), Timestamp('2023-01-15 00:00:00', freq='D'), Timestamp('2023-01-16 00:00:00', freq='D'), Timestamp('2023-01-17 00:00:00', freq='D'), Timestamp('2023-01-18 00:00:00', freq='D'), Timestamp('2023-01-19 00:00:00', freq='D'), Timestamp('2023-01-20 00:00:00', freq='D'), Timestamp('2023-01-21 00:00:00', freq='D'), Timestamp('2023-01-22 00:00:00', freq='D'), Timestamp('2023-01-23 00:00:00'

In [11]:
########################################## ADD initial column MISS into each mentee's file ##########################################

# print(mypath_short)
folder = mypath_short + "Mentees/"

for dfmini in df_master:
    Miss_week_col = []
    for week in dfmini["Week Number"]:
        if week == "nan":
            Miss_week_col.append(week)
        else:
            Miss_week_col.append(False)
    dfmini["Miss"] = Miss_week_col
    filename = folder + str(dfmini["users.id"][0]) + ".xlsx"
    print(filename)
    # dfmini.to_excel(filename)


/Users/lucyl/Desktop/2022_PhD_CompScience/BattlePeer_Research/BattlePeer_Data/2023-07-27/Mentees/58.xlsx
/Users/lucyl/Desktop/2022_PhD_CompScience/BattlePeer_Research/BattlePeer_Data/2023-07-27/Mentees/100.xlsx
/Users/lucyl/Desktop/2022_PhD_CompScience/BattlePeer_Research/BattlePeer_Data/2023-07-27/Mentees/116.xlsx
/Users/lucyl/Desktop/2022_PhD_CompScience/BattlePeer_Research/BattlePeer_Data/2023-07-27/Mentees/141.xlsx
/Users/lucyl/Desktop/2022_PhD_CompScience/BattlePeer_Research/BattlePeer_Data/2023-07-27/Mentees/62.xlsx
/Users/lucyl/Desktop/2022_PhD_CompScience/BattlePeer_Research/BattlePeer_Data/2023-07-27/Mentees/74.xlsx
/Users/lucyl/Desktop/2022_PhD_CompScience/BattlePeer_Research/BattlePeer_Data/2023-07-27/Mentees/54.xlsx
/Users/lucyl/Desktop/2022_PhD_CompScience/BattlePeer_Research/BattlePeer_Data/2023-07-27/Mentees/136.xlsx
/Users/lucyl/Desktop/2022_PhD_CompScience/BattlePeer_Research/BattlePeer_Data/2023-07-27/Mentees/81.xlsx
/Users/lucyl/Desktop/2022_PhD_CompScience/BattlePee

In [12]:
########################################## Compare and add missing dates into each mentee's dataframe ##########################################
# Build date objects list for all dates 
# current_date = datetime.today()
datelist_2023 = pd.date_range(start="2023-01-01",end=today_date).tolist()

datelist_2023_fm = []
for date in datelist_2023:
    date_object = date.strftime('%Y-%m-%d')
    datelist_2023_fm.append(date_object)
# print(datelist_2023_fm)
print(len(datelist_2023_fm))
# print(df_master[35]["Short Date"].tolist())

# mypath_short = "C:/Users/lucyle/Desktop/BattlePeer_json/" + str(today_date) + "/"
# mypath_short = "/Users/lucyl/Desktop/2022 PhD CompScience/iPeer Veterans w PTSD Research/BattlePeer json/" + str(today_date) + "/"
temp_path = "/Users/lucyl/Desktop/2022_PhD_CompScience/BattlePeer_Research/BattlePeer_Data"
mypath_short = temp_path + "/" + str(today_date) + "/"

folder = mypath_short + "Mentees_1/"
os.mkdir(folder)
print("Made folder")

for dfmini in df_master:
    print(dfmini["users.id"][0])

    dates_list2 = []
    for date in dfmini["Short Date"].tolist():
        print(date)
        if str(date) != "" and str(date) != "nan":
            date_object = datetime.strptime(date, '%Y-%m-%d').date()
            dates_list2.append(date_object)

    for date1 in datelist_2023_fm:
        print(date1)
        date_object = datetime.strptime(date1, '%Y-%m-%d').date()
        week_number = date_object.isocalendar()[1]

        if date_object in dates_list2:
            print("It's present")
            # miss = False
            
        else:
            print("It's missing")
            miss = True

            if date1 == "2023-01-01":
                week_number = 0
                
            newrow = pd.DataFrame({"id":dfmini["id"][0], "name":dfmini["name"][0], \
                               "address":dfmini["address"][0], "users.id":dfmini["users.id"][0], \
                                "users.initials":dfmini["users.initials"][0], "Short Date":date1, \
                                    "Week Number":week_number, "Miss":miss}, index=[0])
            dfmini = dfmini.append(newrow, ignore_index=True)

    dfmini["Miss"] = dfmini["Miss"].replace([False, True], ["CheckedIn","Missed"])

    filename = folder + str(dfmini["users.id"][0]) + ".xlsx"
    dfmini.to_excel(filename)


208
Made folder
58
2023-02-17
2023-03-02
2023-03-03
2023-03-06
2023-03-07
2023-03-08
2023-03-13
2023-03-17
2023-03-20
2023-03-23
2023-03-27
2023-04-08
2023-04-18
2023-04-25
2023-05-16
2023-06-08
2023-01-01
It's missing
2023-01-02
It's missing
2023-01-03
It's missing
2023-01-04
It's missing
2023-01-05
It's missing
2023-01-06
It's missing
2023-01-07
It's missing
2023-01-08
It's missing
2023-01-09
It's missing
2023-01-10
It's missing
2023-01-11
It's missing
2023-01-12
It's missing
2023-01-13
It's missing
2023-01-14
It's missing
2023-01-15
It's missing
2023-01-16
It's missing
2023-01-17
It's missing
2023-01-18
It's missing
2023-01-19
It's missing
2023-01-20
It's missing
2023-01-21
It's missing
2023-01-22
It's missing
2023-01-23
It's missing
2023-01-24
It's missing
2023-01-25
It's missing
2023-01-26
It's missing
2023-01-27
It's missing
2023-01-28
It's missing
2023-01-29
It's missing
2023-01-30
It's missing
2023-01-31
It's missing
2023-02-01
It's missing
2023-02-02
It's missing
2023-02-03
It

In [13]:
################## Re-read all files in folder Mentees_new ######################

# "C:\Users\lucyle\Desktop\BattlePeer_json\2023-06-25\Mentees_new"
# Build path
import datetime
import os
import pandas as pd

# today_string = str(datetime.date.today())
# print(today_string)
today_string = str(today_date)

# mentees_path = "C:\\Users\\lucyle\\Desktop\\BattlePeer_json"
# mentees_path = "/Users/lucyl/Desktop/2022 PhD CompScience/iPeer Veterans w PTSD Research/BattlePeer json/"
temp_path = "/Users/lucyl/Desktop/2022_PhD_CompScience/BattlePeer_Research/BattlePeer_Data"
mentees_path = temp_path + "/" + today_string + "/Mentees_1"

print(mentees_path)

# Get all files from folder
files_list = []
files_list = os.listdir(mentees_path)
print(files_list)

# read all files into big list of dataframes
df_master1 = []
for file in files_list:
    newpath = mentees_path + "/" + file
    df = pd.read_excel(newpath)
    df_master1.append(df)


/Users/lucyl/Desktop/2022_PhD_CompScience/BattlePeer_Research/BattlePeer_Data/2023-07-27/Mentees_1
['58.xlsx', '100.xlsx', '116.xlsx', '141.xlsx', '62.xlsx', '74.xlsx', '54.xlsx', '136.xlsx', '81.xlsx', '78.xlsx', '97.xlsx', '79.xlsx', '96.xlsx', '137.xlsx', '80.xlsx', '43.xlsx', '55.xlsx', '75.xlsx', '140.xlsx', '59.xlsx', '52.xlsx', '68.xlsx', '130.xlsx', '87.xlsx', '126.xlsx', '91.xlsx', '106.xlsx', '147.xlsx', '64.xlsx', '72.xlsx', '73.xlsx', '65.xlsx', '146.xlsx', '111.xlsx', '107.xlsx', '127.xlsx', '90.xlsx', '69.xlsx', '131.xlsx', '86.xlsx', '45.xlsx', '93.xlsx', '124.xlsx', '85.xlsx', '132.xlsx', '46.xlsx', '108.xlsx', '50.xlsx', '128.xlsx', '70.xlsx', '89.xlsx', '66.xlsx', '145.xlsx', '112.xlsx', '104.xlsx', '144.xlsx', '88.xlsx', '67.xlsx', '129.xlsx', '71.xlsx', '109.xlsx', '148.xlsx', '84.xlsx', '133.xlsx', '92.xlsx', '125.xlsx', '99.xlsx', '76.xlsx', '138.xlsx', '60.xlsx', '143.xlsx', '114.xlsx', '95.xlsx', '83.xlsx', '134.xlsx', '118.xlsx', '56.xlsx', '82.xlsx', '135.xlsx

In [14]:
# ################################### Make pivot table ###########################################
# import pivot_table
# data_mini_70 = pd.DataFrame
# for dfmini in df_master1:
#     if dfmini["users.id"][0] == 70:
#         data_mini_70 = dfmini

# pivot = data_mini_70.pivot_table(index =["users.id",'Week Number',"Miss"],values =['Miss'],aggfunc ='sum')
# print(pivot)

# pivot.to_excel("C:\\Users\\lucyle\\Desktop\\BattlePeer_json\\pivot.xlsx")

In [15]:
# get all users.id mentee's id  into a list
users_id_list = []
for df in df_master1:
    users_id_list.append(df["users.id"][0])
users_id_list.sort()


In [16]:
# Check on counts of mentors and mentees
print("All mentees' IDs: " + str(users_id_list))
print("Total mentees: " + str(len(users_id_list)))

All mentees' IDs: [43, 45, 46, 50, 52, 54, 55, 56, 58, 59, 60, 61, 62, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 95, 96, 97, 98, 99, 100, 103, 104, 106, 107, 108, 109, 111, 112, 114, 115, 116, 118, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148]
Total mentees: 87


In [17]:
# !pip3 install dash
def find_df1(selected_category):
    data_mini = pd.DataFrame
    for dfmini in df_master1:
        if dfmini["users.id"][0] == selected_category:
            data_mini = dfmini
    return data_mini

In [18]:
    # ##### Test pivot table #######
    # pivot_df1 = find_df1(60)
    # # Create a new pivot table  #columns=["Miss","CheckIn","Missed"],
    # # pivot_df = pivot_df1[["users.id","Week Number","Miss"]]
    
    # pivot_table = pivot_df1.pivot_table(values=["users.id"],columns=["Miss"],\
    #                         index=["Week Number"],aggfunc='count')
    # # print(pivot_df)
    # # pivot_df2 = pivot_table[["Week Number","Miss"]]
    # # print(pivot_table.head(20))

    # print(pivot_table["users.id"])

In [19]:
def addlabels(fig, x,y):
    for i in range(len(x)):
        fig.text(i,y[i],y[i])

In [20]:
##### 7-19-23
import pandas as pd
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output, State
from plotly import graph_objects as go


# Initialize the Dash app
app = dash.Dash(__name__)

# Define the layout
app.layout = html.Div([
    html.H1('Dashboard for Mentees Check-ins'),
    dcc.Dropdown(
        id='dropdown',
        options=[{'label': category, 'value': category} for category in users_id_list],
        value=70
    ),
    dcc.Graph(id='graph')
])

# Define the callback function
@app.callback(
    Output('graph', 'figure'),
    [Input('dropdown', 'value')]
)

def update_graph(selected_category):
    # Filter the data based on the selected category
    filtered_df = find_df1(selected_category)
    index_values = filtered_df["Week Number"]
    filtered_pivot_table = filtered_df.pivot_table(values=["users.id"],columns=["Miss"],\
                            index=["Week Number"],aggfunc='count')
    temp = filtered_pivot_table["users.id"]
    y1 = temp["CheckedIn"]
    y2 = temp["Missed"]

    # Create the bar graph
    fig = {
        'data': [
            go.Bar(
                name="CheckedIn",
                x=filtered_pivot_table.index,
                y=y1,
                offsetgroup=0,
                text=y1,
                textposition="inside",
        ),
            go.Bar(
                name="Missed",
                x=filtered_pivot_table.index,
                y=y2,
                offsetgroup=1,
                text=y2,
                textposition="outside",
        ),
        ],
        'layout': {
            'title': f'Statistics for Mentee number {selected_category} on {today_date}', 
            'xaxis': {'title': 'Week Number'},
            'yaxis': {'title': 'Day count'},
        }
    }

    return fig

# Run the app
if __name__ == '__main__':
    app.run_server(debug=False, host='127.0.0.1', port=8080)




/var/folders/gy/71p7db9d5fd88rfkjdskbpgm0000gn/T/ipykernel_9517/3306480607.py:4: UserWarning: 
The dash_core_components package is deprecated. Please replace
`import dash_core_components as dcc` with `from dash import dcc`
  import dash_core_components as dcc
/var/folders/gy/71p7db9d5fd88rfkjdskbpgm0000gn/T/ipykernel_9517/3306480607.py:5: UserWarning: 
The dash_html_components package is deprecated. Please replace
`import dash_html_components as html` with `from dash import html`
  import dash_html_components as html


In [21]:
############################### Dashboard script 7-24-23 ########################
################## Re-read all files in folder Mentees_new ######################

import datetime
import os
import pandas as pd


today_date = datetime.date.today()
today_string = str(datetime.date.today())

# mentees_path = "C:\\Users\\lucyle\\Desktop\\BattlePeer_json"
# mentees_path = "/Users/lucyl/Desktop/2022 PhD CompScience/iPeer Veterans w PTSD Research/BattlePeer json/"
temp_path = "/Users/lucyl/Desktop/2022_PhD_CompScience/BattlePeer_Research/BattlePeer_Data"
mentees_path = temp_path + "/" + today_string + "/Mentees_1"


print(mentees_path)

# Get all files from folder
files_list = []
files_list = os.listdir(mentees_path)
print(files_list)

# read all files into big list of dataframes
df_master1 = []
for file in files_list:
    newpath = mentees_path + "/" + file
    df = pd.read_excel(newpath)
    df_master1.append(df)


# get all users.id mentee's id  into a list
users_id_list = []
for df in df_master1:
    users_id_list.append(df["users.id"][0])
users_id_list.sort()


# Check on counts of mentors and mentees
print("All mentors' IDs: " + str(users_id_list))
print("Total mentors: " + str(len(users_id_list)))




############################# Dashboard ############################# 
import pandas as pd
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output, State
from plotly import graph_objects as go


def find_df1(selected_category):
    data_mini = pd.DataFrame
    for dfmini in df_master1:
        if dfmini["users.id"][0] == selected_category:
            data_mini = dfmini
    return data_mini

# Initialize the Dash app
app = dash.Dash(__name__)

# Define the layout
app.layout = html.Div([
    html.H1('Dashboard for Mentees Check-ins'),
    dcc.Dropdown(
        id='dropdown',
        options=[{'label': category, 'value': category} for category in users_id_list],
        value=70
    ),
    dcc.Graph(id='graph')
])

# Define the callback function
@app.callback(
    Output('graph', 'figure'),
    [Input('dropdown', 'value')]
)

def update_graph(selected_category):
    # Filter the data based on the selected category
    filtered_df = find_df1(selected_category)
    index_values = filtered_df["Week Number"]
    filtered_pivot_table = filtered_df.pivot_table(values=["users.id"],columns=["Miss"],\
                            index=["Week Number"],aggfunc='count')
    temp = filtered_pivot_table["users.id"]
    y1 = temp["CheckedIn"]
    y2 = temp["Missed"]

    # Create the bar graph
    fig = {
        'data': [
            go.Bar(
                name="Checked",
                x=filtered_pivot_table.index,
                y=y1,
                offsetgroup=0,
                text=y1,
                textposition="inside",
        ),
            go.Bar(
                name="Missed",
                x=filtered_pivot_table.index,
                y=y2,
                offsetgroup=1,
                text=y2,
                textposition="outside",
        ),
        ],
        'layout': {
            'title': f'Statistic for Mentee number {selected_category} on {today_string}', 
            'xaxis': {'title': 'Week Number'},
            'yaxis': {'title': 'Day Count'},
        }
    }

    return fig

# Run the app
if __name__ == '__main__':
    app.run_server(debug=False, host='127.0.0.1', port=8050)



/Users/lucyl/Desktop/2022_PhD_CompScience/BattlePeer_Research/BattlePeer_Data/2023-07-27/Mentees_1
['58.xlsx', '100.xlsx', '116.xlsx', '141.xlsx', '62.xlsx', '74.xlsx', '54.xlsx', '136.xlsx', '81.xlsx', '78.xlsx', '97.xlsx', '79.xlsx', '96.xlsx', '137.xlsx', '80.xlsx', '43.xlsx', '55.xlsx', '75.xlsx', '140.xlsx', '59.xlsx', '52.xlsx', '68.xlsx', '130.xlsx', '87.xlsx', '126.xlsx', '91.xlsx', '106.xlsx', '147.xlsx', '64.xlsx', '72.xlsx', '73.xlsx', '65.xlsx', '146.xlsx', '111.xlsx', '107.xlsx', '127.xlsx', '90.xlsx', '69.xlsx', '131.xlsx', '86.xlsx', '45.xlsx', '93.xlsx', '124.xlsx', '85.xlsx', '132.xlsx', '46.xlsx', '108.xlsx', '50.xlsx', '128.xlsx', '70.xlsx', '89.xlsx', '66.xlsx', '145.xlsx', '112.xlsx', '104.xlsx', '144.xlsx', '88.xlsx', '67.xlsx', '129.xlsx', '71.xlsx', '109.xlsx', '148.xlsx', '84.xlsx', '133.xlsx', '92.xlsx', '125.xlsx', '99.xlsx', '76.xlsx', '138.xlsx', '60.xlsx', '143.xlsx', '114.xlsx', '95.xlsx', '83.xlsx', '134.xlsx', '118.xlsx', '56.xlsx', '82.xlsx', '135.xlsx